<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3

### <center> Автор материала: Латышев Евгений (@EvgeniyLatyshev)

## <center> Индивидуальный проект по анализу данных </center>
### <center> Прогнозирование уровня производства конфет в США </center>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 20, 15
import seaborn as sns
from scipy import stats
import pylab
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from itertools import product
import fbprophet
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute, make_forecasting_frame
import xgboost as xgb
from sklearn.metrics import roc_auc_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

RANDOM_STATE=42

###  Описание набора данных и признаков

Каждый год люди съедают огромное количество конфет, которое сначала необходимо произвести. Очевидно, что уровень производства постоянно меняется. Например, в канун нового года или на хеллоуин съедается гораздо больше сладостей, чем в другие времена года, что вносит элемент сезонности. Также, в долгосрочной перспективе, уровень производства непреклонно растёт вместе с ростом населения, что говорит об имеющейся трендовой составляющей. Эти свойства делают рассматриваемый показатель отличным подопытным с точки зрения анализа временных рядов.

Датасет, полученный с [Kaggle](https://www.kaggle.com/rtatman/us-candy-production-by-month) содержит данные об уровне производства конфет в США с января 1972 по август 2017 года.

Каждая запись характеризуется двумя значениями:
- observation_date - дата в формате YYYY-MM-DD
- IPG3113N - [индустриальный продуктовый индекс](https://fred.stlouisfed.org/series/INDPRO). Это универсальный индекс уровня производства, который измеряется как % от уровня производста 2012 года.

В данной работе рассматривается задача прогнозирования будущего уровня производства конфет по имеющимся предыдущим наблюдениям. В качестве горизонта прогнозирования выбран временной интервал в 24 месяца.

###  Первичный анализ признаков, визуальный анализ признаков, поиск зависимостей

In [ ]:
df = pd.read_csv('../../data/candy_production.csv')

In [ ]:
df.head()

Сперва приведём данные к более удобному виду.

In [ ]:
df.index = pd.to_datetime(df['observation_date'], format='%Y-%m-%d')
df['value'] = df['IPG3113N']
df = df.drop(['observation_date', 'IPG3113N'], axis=1)

In [ ]:
df.head()

Теперь проверим данные на наличие пропусков и выбросов.

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
df.describe()

Как видим, датасет содержит 548 значений в промежутке от 50 до 139. Судя по квантилям, данные относительно чистые и не требуют предобработки.

Далее можно выполнить проверку на нормальность и скошенность. Используем критерий Шапиро-Уилка, Q-Q plot и проверку на скошенность из библиотеки scipy.

In [ ]:
stats.shapiro(df['value'])

In [ ]:
stats.probplot(df['value'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
df['value'].plot(kind='hist', bins=25)

In [ ]:
stats.skewtest(df['value'])

Распределение целевой переменной близко к нормальному распределению, но немного скошено.

Пора посмотреть на график самого временного ряда.

In [ ]:
sns.tsplot(data=df['value'], time=df.index)

Посмотрим, что нам даст группировка наблюдений по месяцам и годам.

In [ ]:
df['year'] = df.index.year
df['month'] = df.index.month

In [ ]:
df.head()

In [ ]:
sns.boxplot(x='year', y='value', data=df)

In [ ]:
sns.boxplot(x='month', y='value', data=df)

Не считая спада 1975 года, количесво произведённых конфет продолжало уверенно расти вплоть до начала нового тысечелетия, что, возможно связано с ростом населения США, которое за эти годы увеличилось с 210 млн. до 282 млн. Однако, начиная с 2000 года наблюдается некая стагнация, выраженная сравнительно небольшим разбросом производства в эти года, а с 2005 года уровень производства и вовсе начинает падать, что может быть связано с затухающим интересом ко сладкому.

Как и ожидалось, перед хэллоуином и новым годом (октябрь-декабрь) количество производимых сладостей существенно выше, чем в остальные месяцы, что вызвано повышенным спросом. 

###  Выбор метрики

Типичные метрики в задаче регрессии:
- MAE
- RMSE
- MAPE
- SMAPE

Последние 2 метрики позволяют уйти от абсолютных величин к процентам, что является существенным преимуществом, когда целевая переменная меняется в большом диапазоне, как в решаемой задаче. SMAPE, вопреки своему названию, асимметрично штрафует за недопредсказания и перепредсказания, поэтому для оценки модели будем использовать MAPE.

В обсуждениях XGBoost на github разработчики уточнили, что алгоритм плохо сходится при выборе MAPE в качестве метрики, поэтому также будем рассчитывать и MAE для дальнейшего сравнения моделей.

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

###  Выбор модели

Наиболее часто для предсказания временных рядов используются следующие методы:
- Простейшие модели (наивная, предсказание средним, экспоненциальное сглаживание и т.п.)
- Линейные модели
- Эконометрические модели (ARIMA)
- Facebook Prophet
- XGBoost
- LSTM RNNs

Простейшие модели, как правило, используются как baseline и дают куда менее точные прогнозы, чем остальные методы.

Линейные модели плохо справляются с нелинейными зависимостями и едва ли дадут хороший результат в решаемой задаче. Разве что, можно воспользоваться регрессией с lasso-регуляризагией для отбора признаков на вход другой модели.

Чаще всего, в статьях на arxiv.org рассматриваемая задача решается с помощью LSTM-нейросеток, но мне пока что не хватает навыков в работе с нейросетями, поэтому данный метод также останется за рамками этой работы.

А вот оставшиеся 3 модели можно проверить на имеющихся данных и выбрать ту, которая себя проявит лучше других.

###  Предобработка данных и создание новых признаков

### ARIMA

Сперва подготовим данные для обучения <b>ARIMA</b>-модели. В основе данного подхода лежит предположение о том, что исследуемый временной ряд стационарен. Если это не так, нужно произвести соответсвующие преобразования, чтобы он стал таковым.

Можно провести STL-декомпозицию ряда и проверить остатки на стационарность с помощью критерия Дики-Фуллера.

In [ ]:
sm.tsa.seasonal_decompose(df['value']).plot()
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['value'])[1])

Критерий Дики-Фуллера отверг гипотезу о стационарности ряда. Следовательно, требуется его дальнейшая обработка.

Продиффериенциируем ряд с лагом 12 (по количеству месяцев).

In [ ]:
df['value_diff'] = df['value'] - df['value'].shift(12)
sm.tsa.seasonal_decompose(df['value_diff'][12:]).plot()
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['value_diff'][12:])[1])

Остатки стационарны, тренд постоянен, можно подавать его на вход <b>ARIMA</b>.

Отложим 2 года для дальнейшей оценки модели.

In [ ]:
df_train = df[:-24]
df_test = df[-24:]

### Facebook Prophet

Главное преимущество <b>Facebook Prophet</b> - простота использования. Временной ряд не нуждается в предобработке и подаётся на вход в виде Pandas DataFrame'а с 2 колонками: 'ds' типа Date/Datetime и 'y' - значением прогнозируемой величины.

In [ ]:
prophetdf = pd.DataFrame()
prophetdf['ds'] = df.index
prophetdf['y'] = df['value'].values

Также отложим 2 года для оценки модели.

In [ ]:
prophetdf_train = prophetdf[:-24]
prophetdf_test = prophetdf[-24:]

### XGBoost

Для <b>XBGoost</b> нужно постараться нагенерировать побольше информативных категориальных и числовых признаков. Например, можно идти по временному ряду окном и считать различные статистики. Вместо того, чтобы делать это самому, а потом выбирать наиболее релевантные фичи из десятков а то и сотен, можно воспользоваться библиотекой <b>tsfresh</b>, которая делает это сама.

In [ ]:
XGBdf, XGBy = make_forecasting_frame(df['value'], kind='kind', max_timeshift=36, rolling_direction=1)

In [ ]:
XGBdf.head()

In [ ]:
XGBX = extract_features(XGBdf, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)

In [ ]:
print(XGBX.shape)

Отбросим константные признаки.

In [ ]:
# drop constant features
XGBX = XGBX.loc[:, XGBX.apply(pd.Series.nunique) != 1] 
print(XGBX.shape)

Добавим предыдущее значение как признак.

In [ ]:
XGBX["feature_last_value"] = XGBy.shift(1)

In [ ]:
XGBX = XGBX.iloc[1:, ]
XGBy = XGBy.iloc[1: ]

In [ ]:
XGBX.head()

Мы получили 377 числовых признаков, характеризующих временной ряд, на которых уже можно обучить <b>XGBoost</b>.

Снова отложим последние 2 года для оценки модели.

In [ ]:
XGBX_train = XGBX[:-24]
XGBy_train = XGBy[:-24]
XGBX_test = XGBX[-24:]
XGBy_test = XGBy[-24:]

###  Кросс-валидация, настройка гиперпараметров модели, построение кривых валидации и обучения

### ARIMA

Очевидно, имеет смысл учесть в модели годовую сезонность - взять разность с лагом 12.

In [ ]:
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(df_train['value_diff'][12:].values.squeeze(), lags=58, ax=ax)
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(df_train['value_diff'][12:].values.squeeze(), lags=58, ax=ax);

Функцию подбора параметров модели можно позаимствовать из лекции. Границы интервалов поиска параметров ARIMA зададим исходя из коррелограмм, на которых явно прослеживается автокорреляция с лагом 1.

In [ ]:
%%time
results = []
best_aic = float("inf")

ps = range(0, 3)
d=1
qs = range(0, 3)
Ps = range(0, 3)
D=1
Qs = range(0, 3)
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)

for param in parameters_list:
    try:
        model=sm.tsa.statespace.SARIMAX(df_train['value'], order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], D, param[3], 12)).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())

Остатки лучшей модели:

In [ ]:
plt.subplot(211)
best_model.resid[13:].plot()
plt.ylabel(u'Остатки')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)

print("Критерий Стьюдента: p=%f" % stats.ttest_1samp(best_model.resid[13:], 0)[1])
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(best_model.resid[13:])[1])

Исходя из критерия Стьюдента, остатки смещены. Однако они стационарны (подтверждается критерием Дики-Фуллера и визуально) и неавтокоррелированы (подтверждается критерием Льюнга-Бокса и коррелограммой). Неидеальный случай, но жить можно.

Посмотрим, насколько хорошо модель описывает временной ряд.

In [ ]:
df_train['model'] = best_model.fittedvalues
df_train['value'].plot()
df_train['model'][13:].plot(color='r')
plt.ylabel('US candy production');

Видим, что модель очень хорошо подогналась под наши данные. Осталось проверить её на отложенной выборке.

# Facebook Prophet

Как уже упоминалось ранее, главное преимущество fbprophet - простота в использовании. Модель не требует настройки гиперпараметров.

In [ ]:
fbmodel = fbprophet.Prophet(mcmc_samples=500)
fbmodel.fit(prophetdf_train)

# XGBoost

Обучим модель из не более чем 1000 итераций. Валидируемся разбиением тренировочной выборки на 5 фолдов.

In [ ]:
dtrain = xgb.DMatrix(XGBX_train, label=XGBy_train)

In [ ]:
params = {
        'objective': 'reg:linear',
        'booster':'gblinear'
    }
rounds = 1000

In [ ]:
cv = xgb.cv(params, dtrain, metrics = ('mae'), verbose_eval=False, nfold=5, 
            show_stdv=False, num_boost_round=rounds, seed=42)

In [ ]:
bst = xgb.train(params, dtrain, num_boost_round=cv['test-mae-mean'].argmin())

Посмотрим MAE на кросс-валидации.

In [ ]:
cv['test-mae-mean'].min()

In [ ]:
prediction_train = bst.predict(dtrain)
plt.plot(prediction_train)
plt.plot(XGBy_train.values)
plt.axis('tight')
plt.grid(True)

###  Прогноз на отложенной выборке

### SARIMAX

In [ ]:
df_train["arima_model"] = best_model.fittedvalues
forecast = best_model.predict(start = df_train.shape[0], end = df_train.shape[0]+24)
forecast = df_train.arima_model.append(forecast).values
forecast = forecast[12:]

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(forecast, color='r', label="model")

plt.title("SARIMA model\n")
plt.plot(df['value'].values[12:], label="actual")
plt.legend()
plt.axvspan(len(df['value'])-36, len(forecast), alpha=0.5, color='lightgrey')
plt.grid(True)

Как видим, модель обманулась скачком значений прямо на конце тренировочного периода и на тесте выдала немного завышенный прогноз.

In [ ]:
mean_absolute_percentage_error(df['value'].values[-24:], forecast[-24:])

In [ ]:
mean_absolute_error(df['value'].values[-24:], forecast[-24:])

MAPE 10.00%, MAE 11.04 на отложенной выборке - довольно неплохой результат.

### Facebook Prophet

In [ ]:
future = fbmodel.make_future_dataframe(periods=24, freq='M')
fbforecast = fbmodel.predict(future)
fbmodel.plot(fbforecast);

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(fbforecast.yhat, color='r', label="model")

plt.title("fbprophet\n")
plt.plot(prophetdf['y'].values[12:], label="actual")
plt.legend()
plt.axvspan(len(df['value'])-36, len(fbforecast), alpha=0.5, color='lightgrey')
plt.grid(True)

In [ ]:
mean_absolute_percentage_error(prophetdf.y[-24:], fbforecast.yhat[-24:])

In [ ]:
mean_absolute_error(prophetdf.y[-24:], fbforecast.yhat[-24:])

Как итог, получаем MAPE в 11.55% и MAE в 12.46 на отложенной выборке. Как видно из графика, модель уловила основной тренд и годовую сезонность, но недостаточно резко реагирует на сильные колебания. Очень неплохой результат, учитывая, какими незначительными усилиями был получен прогноз, но другие модели позволяют достичь лучшего результата за счёт более тонкой настройки.

### XGBoost

In [ ]:
dtest = xgb.DMatrix(XGBX_test)

In [ ]:
prediction_test = bst.predict(dtest)

In [ ]:
prediction = np.append(prediction_train, prediction_test)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(prediction, color='r', label="model")
plt.title("XGBoost\n")
plt.plot(df['value'].values[1:], label="actual")
plt.legend()
plt.axvspan(len(df['value'])-36, len(fbforecast), alpha=0.5, color='lightgrey')
plt.grid(True)

In [ ]:
mean_absolute_error(XGBy_test, prediction_test)

XGBoost наголову обогнал другие методы со вдвое меньшим значением MAE на отложенной выборке.

### Выводы 

В ходе выполнения работы было получено 3 модели, способные делать прогноз уровня производства конфет в США на 2 года вперёд. Как оказалось, довольно неплохой прогноз можно получить почти без усилий при помощи библотки Facebook Prophet, а вот ARIMA требует дополнительной предобработки данных и настройки гиперпараметров. Ну а XGBoost вновь продемонстрировал, за что его так любят на Kaggle.

В дальнейшем, прогнозы можно улучшить за счёт использования новой модели (например, LSTM RNN) или генерации других фич, используемых в обработке сигналов (например, Fast Fourier Transform).